In [ ]:
import copy
import random


groupAPrefers = {
 'ger':  ['qtr', 'eng', 'aus', 'arg', 'jpn', 'usa', 'net', 'fra'],
 'kor':  ["qtr", "eng", "net", "usa", "fra", "jpn", "aus", "arg"],
 'bra':  ["qtr", "eng", "aus", "fra", "arg", "jpn", "usa", "net"],
 'por':  ["fra", "qtr", "eng", "net", "aus", "arg", "jpn", "usa"],
 'spain':  ["fra", "qtr", "eng", "aus", "arg", "net", "jpn", "usa"],
 'can':  ["arg", "aus", "jpn", "eng", "fra", "qtr", "usa", "net"],
 'bel':  ["arg", "aus", "jpn", "fra", "eng", "net", "usa", "qtr"],
 'mor':  ["aus", "eng", "arg", "net", "qtr", "usa", "jpn", "fra"],}
groupBPrefers = {
 'qtr':  ['ger', 'bra', 'spain', 'bel', 'kor', 'por', 'can', 'mor'],
 'jpn':  ["bel", "spain", "kor", "bra", "can", "mor", "por", "ger"],
 'eng':  ["kor", "bel", "mor", "por", "ger", "can", "spain", "bra"],
 'usa':  ["mor", "kor", "spain", "por", "bel", "bra", "can", "ger"],
 'aus':  ["ger", "bra", "spain", "bel", "kor", "por", "can", "mor"],
 'net':  ["ger", "kor", "spain", "mor", "bel", "bra", "can", "por"],
 'arg':  ["bel", "kor", "bra", "ger", "spain", "mor", "por", "can"],
 'fra':  ["ger", "can", "bel", "kor", "por", "bra", "mor", "spain"],}

groupA = sorted(groupAPrefers.keys())
groupB = sorted(groupBPrefers.keys())
print("GroupA: ", groupA)
print("GroupB: ", groupB)

# she  = b
# he = a
# shelikes = blikes
# shelikesbetter = blikesbetter
# helikes = alikes
# helikesbetter = alikesbetter
# guy = teama
# gal = teamb
# guysgirl = teamaxteamb
# guylikes = teamalikes
# girlsguy = teambxteama
# gallikes = teamblikes

def check(fixtures):
    inversefixtures = dict((v,k) for k,v in fixtures.items())
    for b, a in fixtures.items():
        blikes = groupBPrefers[b]
        blikesbetter = blikes[:blikes.index(a)]
        alikes = groupAPrefers[a]
        alikesbetter = alikes[:alikes.index(b)]
        for teama in blikesbetter:
            teamaxteamb = inversefixtures[teama]
            teamalikes = groupAPrefers[teama]
            if teamalikes.index(teamaxteamb) > teamalikes.index(b):
                print("%s and %s like each other better than "
                      "their present opponents: %s and %s, respectively"
                      % (b, teama, a, teamaxteamb))
                return False
        for teamb in alikesbetter:
            teambxteama = fixtures[teamb]
            teamblikes = groupBPrefers[teamb]
            if teamblikes.index(teambxteama) > teamblikes.index(a):
                print("%s and %s like each other better than "
                      "their present opponents: %s and %s, respectively"
                      % (a, teama, b, teambxteama))
                return False
    return True

# guy  = a
# gal = b
# guyslist = alist
# galslist = blist
# fiance = potentialFix

def matchmaker(groupAPrefers, groupBPrefers):
    teamsingroupa = groupA[:]
    fixtures  = {}
    groupAPrefersCopy = copy.deepcopy(groupAPrefers)
    groupBPrefersCopy = copy.deepcopy(groupBPrefers)
    total_matchmakings = 0
    unstable_matchmakings = 0
    while teamsingroupa:
        total_matchmakings+=1
        a = teamsingroupa.pop(0)
        alist = groupAPrefersCopy[a]
        b = alist.pop(0)
        potentialfix = fixtures.get(b)
        if not potentialfix:
            # Opponent is free
            fixtures[b] = a
            print("  %s and %s" % (b, a))
        else:
            # The Opponent is already matched with another team!
            blist = groupBPrefersCopy[b]
            if blist.index(potentialfix) > blist.index(a):
                # She prefers new guy
                unstable_matchmakings+=1
                fixtures[b] = a
                print("  %s dumped %s for %s" % (b, potentialfix, a))
                if groupAPrefersCopy[potentialfix]:
                    # Ex has more girls to try
                    teamsingroupa.append(potentialfix)
            else:
                # She is faithful to old fiance
                if alist:
                    # Look again
                    teamsingroupa.append(a)
    return fixtures, total_matchmakings, unstable_matchmakings


print('\nGroup A proposes a fixture with GroupB:')
fixtures, total_matchmakings_before, unstable_matchmakings_before = matchmaker(groupAPrefers, groupBPrefers)

print('\nFixtures:')
print('  ' + ',\n  '.join('%s is fixed with to %s' % teams
                          for teams in sorted(fixtures.items())))
print()
print('Fixture stability check PASSED'
      if check(fixtures) else 'Fixture stability check FAILED')


# util function to shuffle preflists of groups
def randomizePrefLists(teamPrefs):
    for values in teamPrefs.values():
        # print(values)
        random.shuffle(values)
    return teamPrefs
print()
print('Number of stable pairings BEFORE randomizing: ', len(fixtures))
print('Total Iterations Performed: ', total_matchmakings_before)
print('Total Unstable Matches: ', unstable_matchmakings_before)
print('Percentage of stable playoff matches BEFORE randomizing: ', ((total_matchmakings_before-unstable_matchmakings_before)/total_matchmakings_before)*100, '%')

# randomize preflists for group a and group b 1000 times

for x in range(0,1000):
    randomizePrefLists(groupAPrefers)
    randomizePrefLists(groupBPrefers)
    
# perform Gale Shapley algorithm again
print()
print('Get new matching after randomizing by performing Gale Shapley')
newFixtures, total_matchmakings_after, unstable_matchmakings_after = matchmaker(groupAPrefers, groupBPrefers)
print()
print('\nNew Fixtures:')
print('  ' + ',\n  '.join('%s is fixed with to %s' % teams
                          for teams in sorted(newFixtures.items())))
print('Fixture stability check PASSED'
      if check(newFixtures) else 'Fixture stability check FAILED')
print()
print('Total Iterations Performed: ', total_matchmakings_before)
print('Total Unstable Matches: ', unstable_matchmakings_before)
print('Percentage of stable playoff matches AFTER randomizing: ', ((total_matchmakings_after-unstable_matchmakings_after)/total_matchmakings_after)*100, '%')






In [ ]:
import string
from string import ascii_lowercase, digits 
import random
from random import choice
from ipaddress import collapse_addresses 
from itertools import product
import timeit
import matplotlib.pyplot as plt
random.seed(10)
def stable(ranks, A, B):
    teams = dict((a, (ranks[(a, 1)], 1)) for a in A)
    isStable = False 
    while isStable == False:
        isStable = True 
        for b in B:
            isPaired = False
            for n in range(1, len(B) + 1):
                a = ranks[(b, n)] 
                a_partner, a_n = teams[a] 
                if a_partner == b:
                    if isPaired:
                        isStable = False
                        partners[a] = (ranks[(a, a_n + 1)], a_n + 1)
                    else:
                        isPaired = True
    return sorted((a, b) for (a, (b, n)) in partners.items())
def data_maker(n):
    print('Printing N',n)
    SuperGroup1 = [i for i in range(1,n+1)]
    chars = ascii_lowercase + digits
    SuperGroup2 = [''.join(choice(chars) for _ in range(2)) for _ in range(1,n+1)]
    rank = dict()
    for i in range(len(SuperGroup1)):
        values = [i for i in range(1,n+1)] 
        random.shuffle(values)
        rank[SuperGroup1[i]] = values
    for i in range(len(SuperGroup2)):
        values = [i for i in range(1,n+1)] 
        random.shuffle(values)
        rank[SuperGroup2[i]] = values
    Arankings = dict(((a, rank[a][b_]), SuperGroup2[b_]) for (a, b_) in product(SuperGroup1, range(0, n))) 
    Brankings = dict(((b, rank[b][a_]), SuperGroup1[a_]) for (b, a_) in product(SuperGroup2, range(0, n))) 
    rankings = Arankings
    rankings.update(Brankings)
    return SuperGroup1,SuperGroup2,rankings
def run(): 
    start = 8
    end = 8*16 
    timeComplexity = [] 
    dataComplexity = [] 
    m=1 
    while(start<=end):
        starttime = timeit.default_timer() 
        dataComplexity.append(start) 
        SuperGroup1,SuperGroup2,rankings = data_maker(start)
        (stable(rankings,SuperGroup1,SuperGroup2),start)
        m *= 2
        start = start * m
        tt = timeit.default_timer() - starttime 
        timeComplexity.append(tt)
    print(dataComplexity,timeComplexity) 
    plt.plot(dataComplexity,timeComplexity) 
    plt.title('Time Graph') 
    plt.xlabel('Data')
    plt.ylabel('Time') 
    plt.legend() 
    plt.show()
run()